In [1]:
import shutil
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

In [2]:
import country_converter as coco

cc = coco.CountryConverter()

def country_name_converter(df, col_name):
    some_names = list(df[col_name])
    standard_names = cc.convert(names = some_names, to = 'name_short')
    df['country2'] = standard_names
    df = df.replace('not found', np.nan)
    num_nulls = df['country2'].isnull().sum().sum()
    print('number of nulls:', num_nulls)
    df['country2'] = np.where(df['country2'].isna(), df[col_name], df['country2'])
    df = df.drop(col_name, axis=1)
    df = df.rename(columns={'country2': 'country'})
    return df

In [3]:
covid = pd.read_csv('../data/processed/covid.csv')
df = covid[covid['Date']==max(covid['Date'])]

In [4]:
df = country_name_converter(df, 'Country/Region')

C:\Users\dongt\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


number of nulls: 2


In [5]:
df

,Province/State,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,country
53200,NaN,33.939110,67.709953,8/9/20,37054,1312,25960,9782,Afghanistan
53201,NaN,41.153300,20.168300,8/9/20,6411,199,3342,2870,Albania
53202,NaN,28.033900,1.659600,8/9/20,35160,1302,24506,9352,Algeria
53203,NaN,42.506300,1.521800,8/9/20,955,52,839,64,Andorra
53204,NaN,-11.202700,17.873900,8/9/20,1672,75,567,1030,Angola
...,...,...,...,...,...,...,...,...,...
53461,NaN,0.186400,6.613100,8/9/20,878,15,800,63,Sao Tome and Principe
53462,NaN,15.552727,48.516388,8/9/20,1804,515,913,376,Yemen
53463,NaN,-11.645500,43.333300,8/9/20,399,7,369,23,Comoros
53464,NaN,38.861000,71.276100,8/9/20,7745,62,6484,1199,Tajikistan


## #1. Same-Sex Marriage

In [6]:
ssmarriage = pd.read_csv('../data/processed/same_sex_marriage_poll.csv')

In [7]:
ssmarriage.sample(7)

,Country/Region,Year,For,Against,Neither,country_abbr
33,Germany,2019,84,12.0,4.0,DEU
31,France,2019,79,15.0,6.0,FRA
59,Panama,2017,22,NaN,NaN,PAN
89,Vietnam,2014,34,53.0,13.0,VNM
9,Belgium,2019,82,17.0,1.0,BEL
27,Ecuador,2019,23,51.0,26.0,ECU
57,Nicaragua,2017,25,NaN,NaN,NIC


In [8]:
ssm = country_name_converter(ssmarriage, 'Country/Region')

number of nulls: 0


In [9]:
df_ssm = pd.merge(df, ssm, how='left', on='country')

In [10]:
df = df_ssm

## #2. Divorce data

In [11]:
divorce = pd.read_csv('../data/processed/divorce.csv')

In [12]:
dvc = country_name_converter(divorce, 'Country/Region')

number of nulls: 1


In [13]:
df_ssm_dvc = pd.merge(df, dvc, how='left', on='country')

In [14]:
try:
    os.remove('../data/processed/covid_ssm_dvc.csv')
except:
    pass
df_ssm_dvc.to_csv('../data/processed/covid_ssm_dvc.csv')

In [15]:
df = df_ssm_dvc

## #3 Countries basics

In [16]:
cow = pd.read_csv('../data/processed/countries_basics.csv')
cow.head(2)

,Country/Region,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"


In [17]:
cow = country_name_converter(cow, 'Country/Region')

number of nulls: 2


In [18]:
df_ssm_dvc_cow = pd.merge(df, cow, how='left', on='country')

In [19]:
df = df_ssm_dvc_cow

## #4.  Economy dataset

In [20]:
economy_2015 = pd.read_csv('../data/processed/economy_2015.csv')

In [21]:
df_ssm_dvc_cow_eco = pd.merge(df_ssm_dvc_cow, economy_2015, how='left', on='country')

In [22]:
df = df_ssm_dvc_cow_eco

## #5.  Jobs dataset

In [23]:
jobs_2015 = pd.read_csv('../data/processed/jobs_2015.csv')

In [24]:
jobs_2015.sample(5)

,BM.KLT.DINV.WD.GD.ZS,BM.TRF.PWKR.CD.DT,BX.KLT.DINV.WD.GD.ZS,BX.TRF.PWKR.CD,CM.MKT.LCAP.GD.ZS,EG.ELC.ACCS.ZS,EG.USE.ELEC.KH.PC,EN.POP.DNST,FB.ATM.TOTL.P5,FB.CBK.BRWR.P3,FP.CPI.TOTL,FP.CPI.TOTL.ZG,FS.AST.DOMS.GD.ZS,FS.AST.PRVT.GD.ZS,GB.XPD.RSDV.GD.ZS,GC.TAX.TOTL.GD.ZS,GC.TAX.YPKG.ZS,IC.BUS.EASE.XQ,IC.BUS.NDNS.ZS,IC.BUS.NREG,IC.CRD.INFO.XQ,IC.CRD.PRVT.ZS,IC.CRD.PUBL.ZS,IC.FRM.CORR.ZS,IC.FRM.DURS,IC.ISV.DURS,IC.LGL.DURS,IC.PRP.DURS,IC.REG.DURS,IC.TAX.DURS,IC.TAX.METG,IC.TAX.PAYM,IQ.CPA.ECON.XQ,IQ.CPA.FINS.XQ,IQ.CPA.GNDR.XQ,IQ.CPA.MACR.XQ,IQ.CPA.PROP.XQ,IQ.CPA.PROT.XQ,IQ.CPA.SOCI.XQ,IS.AIR.DPRT,IT.CEL.SETS.P2,IT.MLT.MAIN.P2,IT.NET.BBND.P2,IT.NET.USER.ZS,NE.EXP.GNFS.KD.ZG,NE.EXP.GNFS.ZS,NE.GDI.FPRV.ZS,NE.GDI.TOTL.KD.ZG,NE.GDI.TOTL.ZS,NE.IMP.GNFS.ZS,...,SL.TLF.ADVN.MA.ZS,SL.TLF.ADVN.ZS,SL.TLF.BASC.FE.ZS,SL.TLF.BASC.MA.ZS,SL.TLF.BASC.ZS,SL.TLF.CACT.FE.ZS,SL.TLF.CACT.MA.ZS,SL.TLF.CACT.ZS,SL.TLF.INTM.FE.ZS,SL.TLF.INTM.MA.ZS,SL.TLF.INTM.ZS,SL.TLF.TOTL.IN,SL.UEM.1524.FE.ZS,SL.UEM.1524.MA.ZS,SL.UEM.1524.ZS,SL.UEM.ADVN.FE.ZS,SL.UEM.ADVN.MA.ZS,SL.UEM.ADVN.ZS,SL.UEM.INTM.FE.ZS,SL.UEM.INTM.MA.ZS,SL.UEM.INTM.ZS,SL.UEM.TOTL.FE.ZS,SL.UEM.TOTL.MA.ZS,SL.UEM.TOTL.ZS,SM.POP.TOTL.ZS,SP.ADO.TFRT,SP.DYN.LE00.FE.IN,SP.DYN.LE00.MA.IN,SP.DYN.TFRT.IN,SP.POP.0014.TO,SP.POP.0014.TO.ZS,SP.POP.1564.TO,SP.POP.1564.TO.ZS,SP.POP.65UP.TO,SP.POP.65UP.TO.ZS,SP.POP.DPND,SP.POP.GROW,SP.POP.SCIE.RD.P6,SP.POP.TECH.RD.P6,SP.POP.TOTL,SP.RUR.TOTL,SP.RUR.TOTL.ZS,SP.URB.TOTL,SP.URB.TOTL.IN.ZS,TM.VAL.ICTG.ZS.UN,TX.QTY.MRCH.XD.WD,TX.VAL.FUEL.ZS.UN,TX.VAL.MRCH.XD.WD,TX.VAL.TECH.MF.ZS,country
102,NaN,NaN,NaN,NaN,NaN,37.44632,NaN,209.649672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483.0,12.834775,4.674393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,74.482002,86.955002,80.509003,NaN,NaN,NaN,16033929.0,9.706,12.993,11.364,NaN,NaN,NaN,NaN,NaN,NaN,3.665,5.309,4.523,0.192635,0.2928,74.819,67.829,1.921,5328220.0,21.106946,17467808.0,69.196108,2447889.0,9.696946,44.516799,0.506567,NaN,NaN,25243917.0,9775202.0,38.723,15468715.0,61.277,NaN,435.712197,NaN,586.158192,NaN,North Korea
53,2.408691,2.781262e+09,0.614265,NaN,NaN,100.00000,NaN,135.353251,51.817474,NaN,107.133292,0.452034,214.577516,170.743147,3.01387,33.945947,52.575492,NaN,9.152619,33375.0,6.0,7.7,0.0,NaN,NaN,1.0,485.0,4.0,3.5,130.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.444165,29.831007,42.276164,96.330500,2.347623,55.675573,NaN,0.947854,19.969040,48.383362,...,77.127098,76.060997,52.290798,43.206699,34.573502,57.709999,66.441002,62.023998,70.143303,66.730904,62.799301,2931381.0,10.035,11.528,10.798,4.2932,4.7884,5.1964,4.966,5.3214,5.7789,6.445,5.925,6.170,10.098991,4.2482,82.700,78.800,1.710,957278.0,16.843160,3643647.0,64.109396,1082558.0,19.047444,55.983360,0.706424,7483.57626,NaN,5683483.0,708958.0,12.474,4974525.0,87.526,7.679034,123.887588,4.820216,186.564780,15.962897,Denmark
197,NaN,NaN,11.896121,NaN,NaN,100.00000,NaN,11.842794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8548.0,140.909251,11.643611,0.053906,14.996775,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,53.466000,78.089996,65.435997,NaN,NaN,NaN,2533015.0,8.056,5.407,6.424,NaN,NaN,NaN,NaN,NaN,NaN,3.823,3.396,3.575,3.654712,25.2328,71.167,64.283,2.931,1694303.0,30.444143,3644217.0,65.481240,226764.0,4.074617,52.715494,1.795684,NaN,NaN,5565284.0,2765000.0,49.683,2800284.0,50.317,NaN,227.418244,NaN,558.659218,NaN,Turkmenistan
206,NaN,NaN,0.097757,NaN,NaN,100.00000,NaN,73.575223,10.413845,57.744083,NaN,NaN,NaN,NaN,0.21354,17.467265,26.994550,NaN,0.850360,17785.0,7.0,19.4,0.0,NaN,NaN,2.0,225.0,46.0,5.5,202.0,NaN,58.0,4.0,3.0,4.0,4.0,3.0,3.5,3.7,22579.0,70.323106,9.092875,5.771516,42.800000,-10.290319,19.518322,19.310903,9.600000,23.710903,21.347217,...,NaN,NaN,NaN,NaN,NaN,53.679001,77.453003,65.376999,NaN,NaN,NaN,14721658.0,16.510,14.977,15.559,NaN,NaN,NaN,NaN,NaN,NaN,7.767,8.063,7.939,

In [25]:
df_ssm_dvc_cow_eco_job = pd.merge(df_ssm_dvc_cow_eco, jobs_2015, how='left', on='country')

In [26]:
df = df_ssm_dvc_cow_eco_job

## #6.  Governance dataset

In [27]:
governance_2013 = pd.read_csv('../data/processed/governance_2013.csv')

In [28]:
df_ssm_dvc_cow_eco_job_gov = pd.merge(df_ssm_dvc_cow_eco_job, governance_2013, how='left', on='country')

In [29]:
df_ssm_dvc_cow_eco_job_gov

,Province/State,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,country,Year,For,Against,Neither,country_abbr,Percent,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service,BG.GSR.NFSV.GD.ZS,BM.GSR.CMCP.ZS,BM.GSR.FCTY.CD,BM.GSR.GNFS.CD,BM.GSR.INSF.ZS,BM.GSR.MRCH.CD,BM.GSR.NFSV.CD,BM.GSR.ROYL.CD,BM.GSR.TOTL.CD,BM.GSR.TRAN.ZS,BM.GSR.TRVL.ZS,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS_x,BM.TRF.PRVT.CD,BM.TRF.PWKR.CD.DT_x,BN.CAB.XOKA.CD,...,SP.POP.DPND,SP.POP.GROW,SP.POP.SCIE.RD.P6,SP.POP.TECH.RD.P6,SP.POP.TOTL,SP.RUR.TOTL,SP.RUR.TOTL.ZS,SP.URB.TOTL,SP.URB.TOTL.IN.ZS,TM.VAL.ICTG.ZS.UN,TX.QTY.MRCH.XD.WD,TX.VAL.FUEL.ZS.UN,TX.VAL.MRCH.XD.WD,TX.VAL.TECH.MF.ZS,CC.EST,CC.NO.SRC,CC.PER.RNK,CC.PER.RNK.LOWER,CC.PER.RNK.UPPER,CC.STD.ERR,GE.EST,GE.NO.SRC,GE.PER.RNK,GE.PER.RNK.LOWER,GE.PER.RNK.UPPER,GE.STD.ERR,PV.EST,PV.NO.SRC,PV.PER.RNK,PV.PER.RNK.LOWER,PV.PER.RNK.UPPER,PV.STD.ERR,RL.EST,RL.NO.SRC,RL.PER.RNK,RL.PER.RNK.LOWER,RL.PER.RNK.UPPER,RL.STD.ERR,RQ.EST,RQ.NO.SRC,RQ.PER.RNK,RQ.PER.RNK.LOWER,RQ.PER.RNK.UPPER,RQ.STD.ERR,VA.EST,VA.NO.SRC,VA.PER.RNK,VA.PER.RNK.LOWER,VA.PER.RNK.UPPER,VA.STD.ERR
0,NaN,33.939110,67.709953,8/9/20,37054,1312,25960,9782,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,ASIA (EX. NEAR EAST),31056997.0,647500.0,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38",10.100828,13.681394,1.070221e+08,8.469705e+09,1.227148,7.298363e+09,1.171342e+09,2.424755e+05,8.576727e+09,72.521322,12.570135,2.163053e+06,0.010866,1.300345e+08,2.289918e+08,-4.050307e+09,...,88.768727,2.943234,NaN,NaN,33736494.0,25368831.0,75.197,8367663.0,24.803,0.164524,179.347690,3.450456,416.135516,NaN,-1.436761,11.0,1.895735,0.000000,9.952606,0.160681,-1.384930,8.0,7.582938,0.473934,14.69194,0.220843,-2.519349,6.0,1.421801,0.000000,3.317536,0.245741,-1.597314,12.0,1.408451,0.000000,7.511737,0.159117,-1.193137,9.0,10.90047,5.687204,21.80095,0.176127,-1.240068,12.0,14.553990,8.450705,17.37089,0.123408
1,NaN,41.153300,20.168300,8/9/20,6411,199,3342,2870,Albania,NaN,NaN,NaN,NaN,NaN,NaN,EASTERN EUROPE,3581655.0,28748.0,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579",34.387105,11.122353,2.813238e+08,5.070260e+09,3.162010,3.403345e+09,1.666904e+09,2.457801e+07,5.351583e+09,11.465493,74.249845,7.931056e+07,0.696505,1.573089e+08,1.533124e+08,-9.798251e+08,...,44.016644,-0.291206,NaN,NaN,2880703.0,1226200.0,42.566,1654503.0,57.434,3.583151,533.229073,8.806638,737.967904,1.487346,-0.698603,13.0,27.014220,19.431280,37.914690,0.127816,-0.316920,10.0,45.497630,31.279620,55.45024,0.195889,0.091930,8.0,49.289100,36.492890,62.559240,0.228020,-0.518018,16.0,39.436620,27.699530,45.539910,0.126396,0.209719,12.0,57.81990,51.184830,67.77251,0.168445,0.048893,15.0,51.173710,43.192490,56.33803,0.117732
2,NaN,28.033900,1.659600,8/9/20,35160,1302,24506,9352,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NORTHERN AFRICA,32930091.0,2381740.0,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298",8.727141,57.909666,6.599906e+09,6.329023e+10,2.421260,5.226121e+10,1.102903e+10,2.500106e+08,6.989014e+10,33.460391,6.208682,1.011223e+08,0.060925,2.290666e+08,7.212899e+07,-2.703764e+10,...,52.724919,1.919959,NaN,NaN,39871528.0,11623348.0,29.152,28248180.0,70.848,5.380112,139.275645,94.325860,157.358029,0.151224,-0.473484,14.0,39.336490,26.066350,50.236970,0.149327,-0.534195,11.0,35.071090,22.748820,46.44550,0.174871,-1.202371,7.0,12.796210,7.582938,19.905210,0.231508,-0.689374,16.0,30.985920,19.248830,40.375590,0.133684,-1.165970,9.0,11.84834,5.687204,22.27488,0.183056,-0.893353,16.0,23.943660,15.962440,26.76056,0.110357
3,NaN,42.506300,1.521800,8/9/20,955,52,839,64,Andorra,2013.0,70.0,19.0,11.0,AND,NaN,WESTERN EUROPE,71201.

In [30]:
df = df_ssm_dvc_cow_eco_job_gov

## #7.  Religion dataset

In [31]:
religion_2010 = pd.read_csv('../data/processed/religion_2010.csv')

In [32]:
df_ssm_dvc_cow_eco_job_gov_rlg = pd.merge(df, religion_2010, how='left', on='country')

In [33]:
df = df_ssm_dvc_cow_eco_job_gov_rlg

In [34]:
df

,Province/State,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,country,Year,For,Against,Neither,country_abbr,Percent,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service,BG.GSR.NFSV.GD.ZS,BM.GSR.CMCP.ZS,BM.GSR.FCTY.CD,BM.GSR.GNFS.CD,BM.GSR.INSF.ZS,BM.GSR.MRCH.CD,BM.GSR.NFSV.CD,BM.GSR.ROYL.CD,BM.GSR.TOTL.CD,BM.GSR.TRAN.ZS,BM.GSR.TRVL.ZS,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS_x,BM.TRF.PRVT.CD,BM.TRF.PWKR.CD.DT_x,BN.CAB.XOKA.CD,...,TM.VAL.ICTG.ZS.UN,TX.QTY.MRCH.XD.WD,TX.VAL.FUEL.ZS.UN,TX.VAL.MRCH.XD.WD,TX.VAL.TECH.MF.ZS,CC.EST,CC.NO.SRC,CC.PER.RNK,CC.PER.RNK.LOWER,CC.PER.RNK.UPPER,CC.STD.ERR,GE.EST,GE.NO.SRC,GE.PER.RNK,GE.PER.RNK.LOWER,GE.PER.RNK.UPPER,GE.STD.ERR,PV.EST,PV.NO.SRC,PV.PER.RNK,PV.PER.RNK.LOWER,PV.PER.RNK.UPPER,PV.STD.ERR,RL.EST,RL.NO.SRC,RL.PER.RNK,RL.PER.RNK.LOWER,RL.PER.RNK.UPPER,RL.STD.ERR,RQ.EST,RQ.NO.SRC,RQ.PER.RNK,RQ.PER.RNK.LOWER,RQ.PER.RNK.UPPER,RQ.STD.ERR,VA.EST,VA.NO.SRC,VA.PER.RNK,VA.PER.RNK.LOWER,VA.PER.RNK.UPPER,VA.STD.ERR,christianity_all,judaism_all,islam_all,buddhism_all,zoroastrianism_all,hinduism_all,sikhism_all,confucianism_all,noreligion_all
0,NaN,33.939110,67.709953,8/9/20,37054,1312,25960,9782,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,ASIA (EX. NEAR EAST),31056997.0,647500.0,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38",10.100828,13.681394,1.070221e+08,8.469705e+09,1.227148,7.298363e+09,1.171342e+09,2.424755e+05,8.576727e+09,72.521322,12.570135,2.163053e+06,0.010866,1.300345e+08,2.289918e+08,-4.050307e+09,...,0.164524,179.347690,3.450456,416.135516,NaN,-1.436761,11.0,1.895735,0.000000,9.952606,0.160681,-1.384930,8.0,7.582938,0.473934,14.69194,0.220843,-2.519349,6.0,1.421801,0.000000,3.317536,0.245741,-1.597314,12.0,1.408451,0.000000,7.511737,0.159117,-1.193137,9.0,10.90047,5.687204,21.80095,0.176127,-1.240068,12.0,14.553990,8.450705,17.37089,0.123408,0.000270,0.000000,0.997077,0.000093,0.000074,0.000297,0.000074,0.0,0.002040
1,NaN,41.153300,20.168300,8/9/20,6411,199,3342,2870,Albania,NaN,NaN,NaN,NaN,NaN,NaN,EASTERN EUROPE,3581655.0,28748.0,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579",34.387105,11.122353,2.813238e+08,5.070260e+09,3.162010,3.403345e+09,1.666904e+09,2.457801e+07,5.351583e+09,11.465493,74.249845,7.931056e+07,0.696505,1.573089e+08,1.533124e+08,-9.798251e+08,...,3.583151,533.229073,8.806638,737.967904,1.487346,-0.698603,13.0,27.014220,19.431280,37.914690,0.127816,-0.316920,10.0,45.497630,31.279620,55.45024,0.195889,0.091930,8.0,49.289100,36.492890,62.559240,0.228020,-0.518018,16.0,39.436620,27.699530,45.539910,0.126396,0.209719,12.0,57.81990,51.184830,67.77251,0.168445,0.048893,15.0,51.173710,43.192490,56.33803,0.117732,0.215343,0.000000,0.632678,0.000000,0.000000,0.000000,0.000000,0.0,0.151347
2,NaN,28.033900,1.659600,8/9/20,35160,1302,24506,9352,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NORTHERN AFRICA,32930091.0,2381740.0,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298",8.727141,57.909666,6.599906e+09,6.329023e+10,2.421260,5.226121e+10,1.102903e+10,2.500106e+08,6.989014e+10,33.460391,6.208682,1.011223e+08,0.060925,2.290666e+08,7.212899e+07,-2.703764e+10,...,5.380112,139.275645,94.325860,157.358029,0.151224,-0.473484,14.0,39.336490,26.066350,50.236970,0.149327,-0.534195,11.0,35.071090,22.748820,46.44550,0.174871,-1.202371,7.0,12.796210,7.582938,19.905210,0.231508,-0.689374,16.0,30.985920,19.248830,40.375590,0.133684,-1.165970,9.0,11.84834,5.687204,22.27488,0.183056,-0.893353,16.0,23.943660,15.962440,26.76056,0.110357,0.008016,0.000000,0.991928,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,NaN,42.506300,1.521800,8/9/20,955,52,839,64,Andorra,2013.0,70.0,19.0,11.0,AND,NaN,WESTERN EUROP

In [35]:
df.head(3)

,Province/State,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,country,Year,For,Against,Neither,country_abbr,Percent,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service,BG.GSR.NFSV.GD.ZS,BM.GSR.CMCP.ZS,BM.GSR.FCTY.CD,BM.GSR.GNFS.CD,BM.GSR.INSF.ZS,BM.GSR.MRCH.CD,BM.GSR.NFSV.CD,BM.GSR.ROYL.CD,BM.GSR.TOTL.CD,BM.GSR.TRAN.ZS,BM.GSR.TRVL.ZS,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS_x,BM.TRF.PRVT.CD,BM.TRF.PWKR.CD.DT_x,BN.CAB.XOKA.CD,...,TM.VAL.ICTG.ZS.UN,TX.QTY.MRCH.XD.WD,TX.VAL.FUEL.ZS.UN,TX.VAL.MRCH.XD.WD,TX.VAL.TECH.MF.ZS,CC.EST,CC.NO.SRC,CC.PER.RNK,CC.PER.RNK.LOWER,CC.PER.RNK.UPPER,CC.STD.ERR,GE.EST,GE.NO.SRC,GE.PER.RNK,GE.PER.RNK.LOWER,GE.PER.RNK.UPPER,GE.STD.ERR,PV.EST,PV.NO.SRC,PV.PER.RNK,PV.PER.RNK.LOWER,PV.PER.RNK.UPPER,PV.STD.ERR,RL.EST,RL.NO.SRC,RL.PER.RNK,RL.PER.RNK.LOWER,RL.PER.RNK.UPPER,RL.STD.ERR,RQ.EST,RQ.NO.SRC,RQ.PER.RNK,RQ.PER.RNK.LOWER,RQ.PER.RNK.UPPER,RQ.STD.ERR,VA.EST,VA.NO.SRC,VA.PER.RNK,VA.PER.RNK.LOWER,VA.PER.RNK.UPPER,VA.STD.ERR,christianity_all,judaism_all,islam_all,buddhism_all,zoroastrianism_all,hinduism_all,sikhism_all,confucianism_all,noreligion_all
0,NaN,33.93911,67.709953,8/9/20,37054,1312,25960,9782,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,ASIA (EX. NEAR EAST),31056997.0,647500.0,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38",10.100828,13.681394,1.070221e+08,8.469705e+09,1.227148,7.298363e+09,1.171342e+09,2.424755e+05,8.576727e+09,72.521322,12.570135,2.163053e+06,0.010866,130034470.1,2.289918e+08,-4.050307e+09,...,0.164524,179.347690,3.450456,416.135516,NaN,-1.436761,11.0,1.895735,0.00000,9.952606,0.160681,-1.384930,8.0,7.582938,0.473934,14.69194,0.220843,-2.519349,6.0,1.421801,0.000000,3.317536,0.245741,-1.597314,12.0,1.408451,0.00000,7.511737,0.159117,-1.193137,9.0,10.90047,5.687204,21.80095,0.176127,-1.240068,12.0,14.55399,8.450705,17.37089,0.123408,0.000270,0.0,0.997077,0.000093,0.000074,0.000297,0.000074,0.0,0.002040
1,NaN,41.15330,20.168300,8/9/20,6411,199,3342,2870,Albania,NaN,NaN,NaN,NaN,NaN,NaN,EASTERN EUROPE,3581655.0,28748.0,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579",34.387105,11.122353,2.813238e+08,5.070260e+09,3.162010,3.403345e+09,1.666904e+09,2.457801e+07,5.351583e+09,11.465493,74.249845,7.931056e+07,0.696505,157308927.9,1.533124e+08,-9.798251e+08,...,3.583151,533.229073,8.806638,737.967904,1.487346,-0.698603,13.0,27.014220,19.43128,37.914690,0.127816,-0.316920,10.0,45.497630,31.279620,55.45024,0.195889,0.091930,8.0,49.289100,36.492890,62.559240,0.228020,-0.518018,16.0,39.436620,27.69953,45.539910,0.126396,0.209719,12.0,57.81990,51.184830,67.77251,0.168445,0.048893,15.0,51.17371,43.192490,56.33803,0.117732,0.215343,0.0,0.632678,0.000000,0.000000,0.000000,0.000000,0.0,0.151347
2,NaN,28.03390,1.659600,8/9/20,35160,1302,24506,9352,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NORTHERN AFRICA,32930091.0,2381740.0,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298",8.727141,57.909666,6.599906e+09,6.329023e+10,2.421260,5.226121e+10,1.102903e+10,2.500106e+08,6.989014e+10,33.460391,6.208682,1.011223e+08,0.060925,229066576.8,7.212899e+07,-2.703764e+10,...,5.380112,139.275645,94.325860,157.358029,0.151224,-0.473484,14.0,39.336490,26.06635,50.236970,0.149327,-0.534195,11.0,35.071090,22.748820,46.44550,0.174871,-1.202371,7.0,12.796210,7.582938,19.905210,0.231508,-0.689374,16.0,30.985920,19.24883,40.375590,0.133684,-1.165970,9.0,11.84834,5.687204,22.27488,0.183056,-0.893353,16.0,23.94366,15.962440,26.76056,0.110357,0.008016,0.0,0.991928,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [36]:
df.columns= df.columns.str.lower()

In [38]:
df = df.drop(['province/state', 'year'], axis=1)

In [39]:
# move column 'country' to beginning
li = list(df.columns)
li.remove('country')
li.insert(0, 'country')
df = df[li]
df.sample(3)

,country,lat,long,date,confirmed,deaths,recovered,active,for,against,neither,country_abbr,percent,region,population,area (sq. mi.),pop. density (per sq. mi.),coastline (coast/area ratio),net migration,infant mortality (per 1000 births),gdp ($ per capita),literacy (%),phones (per 1000),arable (%),crops (%),other (%),climate,birthrate,deathrate,agriculture,industry,service,bg.gsr.nfsv.gd.zs,bm.gsr.cmcp.zs,bm.gsr.fcty.cd,bm.gsr.gnfs.cd,bm.gsr.insf.zs,bm.gsr.mrch.cd,bm.gsr.nfsv.cd,bm.gsr.royl.cd,bm.gsr.totl.cd,bm.gsr.tran.zs,bm.gsr.trvl.zs,bm.klt.dinv.cd.wd,bm.klt.dinv.wd.gd.zs_x,bm.trf.prvt.cd,bm.trf.pwkr.cd.dt_x,bn.cab.xoka.cd,bn.cab.xoka.gd.zs,bn.fin.totl.cd,...,tm.val.ictg.zs.un,tx.qty.mrch.xd.wd,tx.val.fuel.zs.un,tx.val.mrch.xd.wd,tx.val.tech.mf.zs,cc.est,cc.no.src,cc.per.rnk,cc.per.rnk.lower,cc.per.rnk.upper,cc.std.err,ge.est,ge.no.src,ge.per.rnk,ge.per.rnk.lower,ge.per.rnk.upper,ge.std.err,pv.est,pv.no.src,pv.per.rnk,pv.per.rnk.lower,pv.per.rnk.upper,pv.std.err,rl.est,rl.no.src,rl.per.rnk,rl.per.rnk.lower,rl.per.rnk.upper,rl.std.err,rq.est,rq.no.src,rq.per.rnk,rq.per.rnk.lower,rq.per.rnk.upper,rq.std.err,va.est,va.no.src,va.per.rnk,va.per.rnk.lower,va.per.rnk.upper,va.std.err,christianity_all,judaism_all,islam_all,buddhism_all,zoroastrianism_all,hinduism_all,sikhism_all,confucianism_all,noreligion_all
115,France,14.641500,-61.024200,8/9/20,336,16,98,222,79.0,15.0,6.0,FRA,54.29,WESTERN EUROPE,60876136.0,547030.0,"111,3","0,63","0,66","4,26",27600.0,"99,0","586,4","33,53","2,07","64,4",4,"11,99","9,14","0,022","0,214","0,764",20.052926,58.362553,1.274900e+11,7.873780e+11,5.390591,5.540260e+11,2.333510e+11,1.566039e+10,9.148690e+11,19.293912,16.936170,5.123076e+10,2.101165,4.088016e+10,1.279155e+10,-9.130437e+09,-0.374473,-1.248698e+09,...,6.673410,97.102804,3.037859,154.694430,26.847247,1.329576,10.0,88.15166,81.51659,90.99526,0.146783,1.481829,7.0,89.099530,82.464450,97.63033,0.216755,0.448744,9.0,62.08531,48.81517,71.56398,0.225787,1.426773,12.0,89.201880,85.915490,92.48827,0.152922,1.157089,8.0,85.308060,75.355450,91.94313,0.223259,1.219247,13.0,89.671360,81.220660,93.896710,0.128818,0.708160,0.007721,0.079626,0.010468,0.0,0.000820,0.000000,0.000000,0.193172
100,Equatorial Guinea,1.650800,10.267900,8/9/20,4821,83,2182,2556,NaN,NaN,NaN,NaN,NaN,SUB-SAHARAN AFRICA,540109.0,28051.0,"19,3","1,06",0,"85,13",2700.0,"85,7","18,5","4,63","3,57","91,8",2,"35,59","15,06","0,03","0,906","0,062",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,301.983443,NaN,592.525068,NaN,-1.593720,7.0,0.00000,0.00000,9.00474,0.222014,-1.541717,6.0,3.317536,0.473934,11.37441,0.205553,0.120355,4.0,51.18483,36.49289,64.45498,0.261303,-1.349341,10.0,7.042253,0.938967,13.14554,0.151278,-1.426211,6.0,6.635071,2.369668,13.27014,0.204977,-1.969891,11.0,1.877934,0.938967,3.755868,0.135694,0.920418,0.000000,0.040634,0.000000,0.0,0.000516,0.000000,0.000000,0.035462
139,Japan,36.204824,138.252924,8/9/20,48782,1047,32867,14868,51.0,41.0,8.0,JPN,35.42,ASIA (EX. NEAR EAST),127463611.0,377835.0,"337,4","7,87",0,"3,26",28200.0,"99,0","461,2","12,19","0,96","86,85",3,"9,37","9,16","0,017","0,258","0,725",7.775119,62.041221,7.264953e+10,8.079870e+11,6.039570,6.293730e+11,1.786140e+11,1.703376e+10,8.806360e+11,22.976154,8.943055,1.384150e+11,3.153338,2.578094e+10,4.000341e+09,1.364720e+11,3.109068,1.809490e+11,...,12.790536,104.658390,1.771475,130.355171,16.782062,1.655680,11.0,92.41706,89.57346,94.78673,0.138034,1.618066,7.0,93.838860,86.255920,98.57820,0.216755,1.019142,9.0,82.93839,66.82465,97.15639,0.225787,1.443707,12.0,90.140850,85.915490,92.48827,0.152922,1.115993,8.0,82.938390,73.933650,90.99526,0.223259,1.109647,12.0,84.976520,73.708920,92.018780,0.130399,0.024275,0.000010,0.001893,0.854822,0.0,0.000228,0.000019,0.001165,0.117567
